In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import sqlite3
import SniperUtils as SU

In [2]:
def filter_opp(opps, min_qty, min_ticks, product = ''):
    filt_opps = opps.copy()
    if product != '':
        filt_opps = filt_opps[filt_opps['poly'].str.contains(product)]
    # remove possibly shared implied markets
    filt_opps = filt_opps[(filt_opps.is_direct==1) | (filt_opps.entry_qty >= 3)]
    # applies the argument qty and edge
    filt_opps = filt_opps[(filt_opps.entry_qty >= min_qty) & (filt_opps.entry_ticks >= min_ticks)]
    filt_opps = filt_opps.rename_axis(None)
    return filt_opps

In [3]:
def max_edge (opp_rows_arg):
    opp_rows = opp_rows_arg
    pos_lsn_win_opp_rows = opp_rows_arg[opp_rows.lsn_win > 0]
    # within a group there can be both -ve and positive lsn_wins
    if pos_lsn_win_opp_rows.shape[0] > 0:
        opp_rows = pos_lsn_win_opp_rows
    m_idx = opp_rows['tot_ticks'].argmax()
    r = opp_rows.iloc[m_idx]
    return r
def least_penalty(opp_rows):
    m_idx = abs(opp_rows['tot_ticks']).argmin()
    r = opp_rows.iloc[m_idx]
    return r

In [4]:
def get_opps(file, min_qty, min_edge, product):
    res = SU.get_sniper_opps(file)
    res = filter_opp(res, min_qty, min_edge, product)
    res = res.groupby('opp_key').apply(lambda x: max_edge(x))
    return res

In [159]:
res = get_opps('/home/taha/spartan_store/sniper_opps/0801-0805/sim_livestocks.csv',1,0,'HE')
res.shape

(14349, 49)

In [137]:
res.columns

Index(['market_date', 'run_id', 'opp_id', 'eid', 'ht_time', 't_time', 'side',
       'poly', 'has_futures', 'min_days', 'max_days', 'min_fut_vol',
       'entry_ticks', 'entry_pnl', 'entry_qty', 'is_direct', 'fsn_win',
       'lsn_win', 'fsn_win_dir', 'lsn_win_dir', 'cp1_win', 'cp1_pbook',
       'cp1_pbook_dir', 'cp2_win', 'cp2_pbook', 'cp2_pbook_dir',
       'penalty_poly', 'penalty_legs', 'entry_ev', 'entry_pbook',
       'entry_pbook_dir', 'entry_leg_books', 'exit_event', 'exit_pbook',
       'exit_pbook_dir', 'exit_leg_books', 'observed_edge_period',
       'first_edge_cognizant_event_ls', 'edge_cognizant_win_ls',
       'n_edge_cognizant_events_ls', 'first_edge_cognizant_event_fs',
       'edge_cognizant_win_fs', 'n_edge_cognizant_events_fs',
       'same_security_opp', 'same_security_opp_pbook',
       'same_security_opp_pbook_zero', 'entry_pbook_zero', 'opp_key',
       'tot_ticks'],
      dtype='object', name=200)

In [146]:
res['poly'].head(10)

opp_key
2021-08-01_10    +(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...
2021-08-01_11    +(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...
2021-08-01_13    +(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...
2021-08-01_14               +(HGU1-HGH2) -(HGU1-HGK2) +(HGH2-HGK2)
2021-08-01_19               +(HGU1-HGZ1) -(HGU1-HGH2) +(HGZ1-HGH2)
2021-08-01_20               +(HGU1-HGV1) -(HGU1-HGZ1) +(HGV1-HGZ1)
2021-08-01_21    +(HGU1-HGV1) -(HGU1-HGX1) +(HGV1-HGZ1) -(HGX1-...
2021-08-01_22    +(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...
2021-08-01_23               +(HGV1-HGX1) -(HGV1-HGZ1) +(HGX1-HGZ1)
2021-08-01_25               +(HGV1-HGX1) -(HGV1-HGZ1) +(HGX1-HGZ1)
Name: poly, dtype: object

In [176]:
res_2ticks = res[res['entry_ticks']>=10]
print(res_2ticks.shape)
same_sec = res_2ticks[res_2ticks['same_security_opp'] == 1]
print(same_sec.shape)
non_same_sec = res_2ticks[res_2ticks['same_security_opp'] != 1]
print(non_same_sec.shape)

(46, 49)
(45, 49)
(1, 49)


In [174]:
non_same_sec[['lsn_win','entry_qty','entry_ticks','min_days','max_days','penalty_poly','penalty_legs','entry_pbook_zero','same_security_opp_pbook_zero']].describe()

,lsn_win,entry_qty,entry_ticks,min_days,max_days,penalty_poly,penalty_legs,entry_pbook_zero,same_security_opp_pbook_zero
count,1.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.190960e+11,1.0,24.0,8.0,313.0,25.0,-1450.0,3.0,4.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.190960e+11,1.0,24.0,8.0,313.0,25.0,-1450.0,3.0,4.0
25%,1.190960e+11,1.0,24.0,8.0,313.0,25.0,-1450.0,3.0,4.0
50%,1.190960e+11,1.0,24.0,8.0,313.0,25.0,-1450.0,3.0,4.0
75%,1.190960e+11,1.0,24.0,8.0,313.0,25.0,-1450.0,3.0,4.0
max,1.190960e+11,1.0,24.0,8.0,313.0,25.0,-1450.0,3.0,4.0


In [175]:
same_sec[['lsn_win','entry_qty','entry_ticks','min_days','max_days','penalty_poly','penalty_legs','entry_pbook_zero','same_security_opp_pbook_zero']].describe()

,lsn_win,entry_qty,entry_ticks,min_days,max_days,penalty_poly,penalty_legs,entry_pbook_zero,same_security_opp_pbook_zero
count,4.500000e+01,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000
mean,2.187314e+10,1.088889,39.244444,12.177778,245.733333,221.111111,-374.444444,7.044444,8.444444
std,7.863225e+10,0.468179,22.983349,27.721250,66.679150,106.594436,1454.338034,2.315516,2.291839
min,-5.225775e+06,1.000000,10.000000,8.000000,132.000000,75.000000,-9900.000000,3.000000,5.000000
25%,-2.628780e+05,1.000000,16.000000,8.000000,193.000000,150.000000,-225.000000,5.000000,6.000000
50%,4.178000e+04,1.000000,46.000000,8.000000,193.000000,175.000000,-125.000000,7.000000,8.000000
75%,1.727842e+07,1.000000,61.000000,8.000000,313.000000,250.000000,-100.000000,9.000000,10.000000
max,3.163746e+11,4.000000,81.000000,194.000000,373.000000,575.000000,-50.000000,13.000000,16.000000


In [158]:
same_sec[['ht_time','side','poly','entry_ticks','same_security_opp_pbook','same_security_opp_pbook_zero','lsn_win']]

,ht_time,side,poly,entry_ticks,same_security_opp_pbook,same_security_opp_pbook_zero,lsn_win
opp_key,,,,,,,
2021-08-01_10,12:51:44.599999169,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0,"Buy: 10 (2), 5 (1), 0 (1), -5 (1), -470 (1) <-...",5.0,26737.0
2021-08-01_20,15:11:52.778107789,Sell,+(HGU1-HGV1) -(HGU1-HGZ1) +(HGV1-HGZ1),2.0,"Buy: 10 (2), 5 (3), -10 (1), -20 (2), -25 (1) ...",5.0,95059.0
2021-08-01_22,15:14:16.636015307,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,3.0,"Buy: 15 (1), 10 (1), 0 (2), -5 (1), -10 (2) <-...",5.0,5782614.0
2021-08-01_23,15:14:16.636723819,Sell,+(HGV1-HGX1) -(HGV1-HGZ1) +(HGX1-HGZ1),2.0,"Buy: 10 (1), 0 (1), -5 (2), -15 (3), -20 (3) <...",4.0,42363.0
2021-08-01_25,15:14:16.638143249,Sell,+(HGV1-HGX1) -(HGV1-HGZ1) +(HGX1-HGZ1),2.0,"Buy: 10 (1), 0 (1), -5 (2), -15 (3), -20 (3) <...",4.0,536366.0
2021-08-01_40,15:17:30.933114395,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0,"Buy: 10 (2), 5 (2), -5 (1), -15 (5), -20 (1) <...",4.0,121972.0
2021-08-01_49,15:29:13.111844593,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0,"Buy: 10 (1), 5 (1), 0 (6), -5 (2), -20 (1) <--...",10.0,1287391.0
2021-08-01_50,15:31:31.168844257,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0,"Buy: 10 (1), 5 (1), 0 (6), -5 (4), -115 (1) <-...",12.0,3755676.0
2021-08-02_52,16:29:57.890770291,Buy,+(HGU1-HGH2) -(HGU1-HGK2) +(HGH2-HGK2),3.0,"Buy: -35 (3), -40 (1), -45 (4), -50 (3), -55 (...",22.0,-364039.0


In [156]:
res_2ticks[['market_date', 'ht_time', 'side','poly','entry_ticks']]

,market_date,ht_time,side,poly,entry_ticks
opp_key,,,,,
2021-08-01_10,2021-08-01,12:51:44.599999169,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0
2021-08-01_20,2021-08-01,15:11:52.778107789,Sell,+(HGU1-HGV1) -(HGU1-HGZ1) +(HGV1-HGZ1),2.0
2021-08-01_22,2021-08-01,15:14:16.636015307,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,3.0
2021-08-01_23,2021-08-01,15:14:16.636723819,Sell,+(HGV1-HGX1) -(HGV1-HGZ1) +(HGX1-HGZ1),2.0
2021-08-01_25,2021-08-01,15:14:16.638143249,Sell,+(HGV1-HGX1) -(HGV1-HGZ1) +(HGX1-HGZ1),2.0
2021-08-01_40,2021-08-01,15:17:30.933114395,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0
2021-08-01_45,2021-08-01,15:20:30.460556929,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0
2021-08-01_49,2021-08-01,15:29:13.111844593,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0
2021-08-01_50,2021-08-01,15:31:31.168844257,Sell,+(HGU1-HGV1) +(HGV1-HGX1) -(HGU1-HGZ1) +(HGX1-...,2.0


In [ ]:
#res = res[res.entry_ticks < 100]
res[res.lsn_win < 55000][['lsn_win','entry_qty','entry_ticks','min_days','max_days','npacks','ntrades','proc_time','fs_adv','fs_adv_ratio','adv_time_per_qty','penalty_poly']].describe()

In [ ]:
# stay longer
res[res.lsn_win >= 55000][['lsn_win','entry_qty','entry_ticks','min_days','max_days','npacks','ntrades','proc_time','fs_adv','fs_adv_ratio','adv_time_per_qty','penalty_poly']].describe()

In [ ]:
res[res.lsn_win >= 500000][['lsn_win','entry_qty','entry_ticks','min_days','max_days','npacks','ntrades','proc_time','fs_adv','fs_adv_ratio','adv_time_per_qty','penalty_poly']].describe()

In [ ]:
res[res.penalty_poly <= 1][['lsn_win','entry_qty','entry_ticks','min_days','max_days','npacks','ntrades','proc_time','fs_adv','fs_adv_ratio','adv_time_per_qty','penalty_poly']].describe()

In [ ]:
stickies = res[res.lsn_win >= 500000]
stickies[stickies.lsn_win <= 600000]